In [44]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageIdCreateEntry, Region, ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

endpoint = "https://b03mlcv20240717.cognitiveservices.azure.com/"
prediction_endpoint = 'https://b03mlcv20240717-prediction.cognitiveservices.azure.com/'
training_key = "10f02e33d99848a980dca38210d41bbf"
prediction_key = "b0f7d07bbe674601aa2aadba2fb994c2"
prediction_resource_id = "/subscriptions/b850d62a-25fe-4d3a-9697-ea40449528a9/resourceGroups/B03-MLDTest/providers/Microsoft.CognitiveServices/accounts/B03MLCV20240717-Prediction"


credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

In [45]:
publish_iteration_name = 'b03-kitchen-v1'
project_name= 'b03-customvision3'

obj_detection_domain = None
for domain in trainer.get_domains():
    if domain.type == 'ObjectDetection' and domain.name =='General':
        obj_detection_domain = domain
        break

print(obj_detection_domain)

{'additional_properties': {}, 'id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1', 'name': 'General', 'type': 'ObjectDetection', 'exportable': False, 'enabled': True, 'exportable_platforms': [], 'model_information': None}


 ### Create Project

In [19]:
def get_or_create_project(trainer, project_name, domain_id):
    # 모든 프로젝트를 가져오기
    all_projects = trainer.get_projects()

    # 동일한 이름의 프로젝트가 있는지 확인
    for project in all_projects:
        if project.name == project_name:
            print(f"기존 프로젝트 '{project_name}'를 찾았습니다.")
            return project

    # 동일한 이름의 프로젝트가 없으면 새로 생성
    print(f"프로젝트 '{project_name}'를 찾을 수 없습니다. 새로 생성합니다.")
    project = trainer.create_project(project_name, domain_id=domain_id)
    return project

# 프로젝트 가져오기 또는 생성하기
project = get_or_create_project(trainer, project_name, domain.id)

#project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

기존 프로젝트 'b03-customvision3'를 찾았습니다.


In [4]:
fork_tag = trainer.create_tag(project_id=project.id, name='fork')
scissors_tag = trainer.create_tag(project.id,'scissors')

In [5]:
fork_image_regions = {
    "fork_1": [ 0.145833328, 0.3509314, 0.5894608, 0.238562092 ],
    "fork_2": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "fork_3": [ 0.09191177, 0.0682516545, 0.757352948, 0.6143791 ],
    "fork_4": [ 0.254901975, 0.185898721, 0.5232843, 0.594771266 ],
    "fork_5": [ 0.2365196, 0.128709182, 0.5845588, 0.71405226 ],
    "fork_6": [ 0.115196079, 0.133611143, 0.676470637, 0.6993464 ],
    "fork_7": [ 0.164215669, 0.31008172, 0.767156839, 0.410130739 ],
    "fork_8": [ 0.118872553, 0.318251669, 0.817401946, 0.225490168 ],
    "fork_9": [ 0.18259804, 0.2136765, 0.6335784, 0.643790841 ],
    "fork_10": [ 0.05269608, 0.282303959, 0.8088235, 0.452614367 ],
    "fork_11": [ 0.05759804, 0.0894935, 0.9007353, 0.3251634 ],
    "fork_12": [ 0.3345588, 0.07315363, 0.375, 0.9150327 ],
    "fork_13": [ 0.269607842, 0.194068655, 0.4093137, 0.6732026 ],
    "fork_14": [ 0.143382356, 0.218578458, 0.7977941, 0.295751631 ],
    "fork_15": [ 0.19240196, 0.0633497, 0.5710784, 0.8398692 ],
    "fork_16": [ 0.140931368, 0.480016381, 0.6838235, 0.240196079 ],
    "fork_17": [ 0.305147052, 0.2512582, 0.4791667, 0.5408496 ],
    "fork_18": [ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
    "fork_19": [ 0.219362751, 0.141781077, 0.5919118, 0.6683006 ],
    "fork_20": [ 0.180147052, 0.239820287, 0.6887255, 0.235294119 ]
}
 
scissors_image_regions = {
    "scissors_1": [ 0.4007353, 0.194068655, 0.259803921, 0.6617647 ],
    "scissors_2": [ 0.426470578, 0.185898721, 0.172794119, 0.5539216 ],
    "scissors_3": [ 0.289215684, 0.259428144, 0.403186262, 0.421568632 ],
    "scissors_4": [ 0.343137264, 0.105833367, 0.332107842, 0.8055556 ],
    "scissors_5": [ 0.3125, 0.09766343, 0.435049027, 0.71405226 ],
    "scissors_6": [ 0.379901975, 0.24308826, 0.32107842, 0.5718954 ],
    "scissors_7": [ 0.341911763, 0.20714055, 0.3137255, 0.6356209 ],
    "scissors_8": [ 0.231617644, 0.08459154, 0.504901946, 0.8480392 ],
    "scissors_9": [ 0.170343131, 0.332957536, 0.767156839, 0.403594762 ],
    "scissors_10": [ 0.204656869, 0.120539248, 0.5245098, 0.743464053 ],
    "scissors_11": [ 0.05514706, 0.159754932, 0.799019635, 0.730392158 ],
    "scissors_12": [ 0.265931368, 0.169558853, 0.5061275, 0.606209159 ],
    "scissors_13": [ 0.241421565, 0.184264734, 0.448529422, 0.6830065 ],
    "scissors_14": [ 0.05759804, 0.05027781, 0.75, 0.882352948 ],
    "scissors_15": [ 0.191176474, 0.169558853, 0.6936275, 0.6748366 ],
    "scissors_16": [ 0.1004902, 0.279036, 0.6911765, 0.477124184 ],
    "scissors_17": [ 0.2720588, 0.131977156, 0.4987745, 0.6911765 ],
    "scissors_18": [ 0.180147052, 0.112369314, 0.6262255, 0.6666667 ],
    "scissors_19": [ 0.333333343, 0.0274019931, 0.443627447, 0.852941155 ],
    "scissors_20": [ 0.158088237, 0.04047389, 0.6691176, 0.843137264 ]
}

 ### 업로드

In [6]:
base_image_location = os.path.join('images')

In [13]:
tagged_images_with_regions = []

for file_name in fork_image_regions.keys():
    x,y,w,h = fork_image_regions[file_name]
    regions = [Region(tag_id=fork_tag.id, left=x,top=y,width=w,height=h)]
    
    with open(os.path.join(base_image_location,'fork','{}.jpg'.format(file_name)),'rb')as image:
        image_data=image.read()
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_data, regions=regions))


In [15]:
print(len(tagged_images_with_regions))

20


In [16]:
upload_result = trainer.create_images_from_files(project_id=project.id, batch=ImageFileCreateBatch(images=tagged_images_with_regions))

if upload_result.is_batch_successful:
    print('succed')
else:
    for image in upload_result.images:
        print('image status: ', image.status)

succed


In [20]:
for file_name in scissors_image_regions.keys():
    x,y,w,h = scissors_image_regions[file_name]
    regions = [Region(tag_id=scissors_tag.id, left=x,top=y,width=w,height=h)]
    
    with open(os.path.join(base_image_location,'scissors','{}.jpg'.format(file_name)),'rb')as image:
        image_data=image.read()
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_data, regions=regions))

upload_result = trainer.create_images_from_files(project_id=project.id, batch=ImageFileCreateBatch(images=tagged_images_with_regions))

if upload_result.is_batch_successful:
    print('succed')
else:
    for image in upload_result.images:
        print('image status: ', image.status)

image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
image status:  OKDuplicate
i

In [21]:
iteration = trainer.train_project(project_id=project.id)

while iteration.status != 'Completed':
    iteration = trainer.get_iteration(project.id, iteration.id)
    print('Train Status: ' + iteration.status)
    time.sleep(3)
    
print('Completed')

Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Status: Training
Train Statu

 ### Publish

In [46]:
#publish
publish = trainer.publish_iteration(project_id=project.id, iteration_id=iteration.id, publish_name=publish_iteration_name, prediction_id=prediction_resource_id)
print(publish)

True


In [50]:
with open(os.path.join(base_image_location,'test','test_image.jpg'),'rb') as image:
    image_data=image.read()
response = predictor.detect_image(project_id=project.id, published_name=publish_iteration_name, image_data=image_data)


In [58]:
for prediction in response.predictions:
    if prediction.probability>0.05:
        print(prediction.tag_name, '{:.2f}'.format(prediction.probability*100))

fork 81.24
